<a href="https://colab.research.google.com/github/tlatkowski/inpainting-gmcnn-keras/blob/master/colab/Image_Inpainting_with_GMCNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download the current version of GMCNN pipeline from GitHub

In [0]:
!git clone https://github.com/tlatkowski/inpainting-gmcnn-keras.git

In [0]:
!ls

# Download and extract NVIDIA's testing mask dataset

In [0]:
!wget http://masc.cs.gmu.edu/wiki/uploads/partialconv/mask.zip
!unzip -q mask.zip

In [0]:
!ls

# Download and extract dataset with training images (Places356)

In [0]:
!wget http://data.csail.mit.edu/places/places365/val_large.tar
!tar -xf val_large.tar
!mkdir images 
!cp -a val_large/ images

In [0]:
!ls

# Install all requirements

In [0]:
!pip install -r inpainting-gmcnn-keras/requirements/requirements.txt

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

In [0]:
LOG_DIR = './outputs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
!mkdir config
!cp inpainting-gmcnn-keras/config/main_config.ini config

In [0]:
%%writefile config/main_config.ini
[TRAINING]
WGAN_TRAINING_RATIO = 1
NUM_EPOCHS = 5
BATCH_SIZE = 4
IMG_HEIGHT = 256
IMG_WIDTH = 256
NUM_CHANNELS = 3
LEARNING_RATE = 0.0001
SAVE_MODEL_STEPS_PERIOD = 1000

[MODEL]
ADD_MASK_AS_GENERATOR_INPUT = True
GRADIENT_PENALTY_LOSS_WEIGHT = 10
ID_MRF_LOSS_WEIGHT = 0.05
ADVERSARIAL_LOSS_WEIGHT = 0.001
NN_STRETCH_SIGMA = 0.5
VGG_16_LAYERS = 3,6,10
ID_MRF_STYLE_WEIGHT = 1.0
ID_MRF_CONTENT_WEIGHT = 1.0
NUM_GAUSSIAN_STEPS = 3
GAUSSIAN_KERNEL_SIZE = 32
GAUSSIAN_KERNEL_STD = 40.0

In [0]:
!ls

# Train generator with only confidence reconstruction loss for 5 epochs

In [0]:
!python inpainting-gmcnn-keras/runner.py --train_path images --mask_path mask --experiment_name "gmcnn256x256" -warm_up_generator

# Visualize predicted images for specific training steps in warm-up generator mode

In [0]:
!ls outputs/gmcnn256x256/predicted_pics/warm_up_generator/

In [0]:
from IPython.display import Image
Image('outputs/gmcnn256x256/predicted_pics/warm_up_generator/step_3000.png')

In [0]:
Image('outputs/gmcnn256x256/predicted_pics/warm_up_generator/step_5000.png')

# Full Wasserstein GAN training mode: generator, local and global discriminators

In [0]:
%%writefile config/main_config.ini
[TRAINING]
WGAN_TRAINING_RATIO = 5
NUM_EPOCHS = 5
BATCH_SIZE = 4
IMG_HEIGHT = 256
IMG_WIDTH = 256
NUM_CHANNELS = 3
LEARNING_RATE = 0.0002
SAVE_MODEL_STEPS_PERIOD = 500

[MODEL]
ADD_MASK_AS_GENERATOR_INPUT = True
GRADIENT_PENALTY_LOSS_WEIGHT = 10
ID_MRF_LOSS_WEIGHT = 0.05
ADVERSARIAL_LOSS_WEIGHT = 0.0005
NN_STRETCH_SIGMA = 0.5
VGG_16_LAYERS = 3,6,10
ID_MRF_STYLE_WEIGHT = 1.0
ID_MRF_CONTENT_WEIGHT = 1.0
NUM_GAUSSIAN_STEPS = 3
GAUSSIAN_KERNEL_SIZE = 32
GAUSSIAN_KERNEL_STD = 40.0

In [0]:
!python inpainting-gmcnn-keras/runner.py --train_path images --mask_path mask -from_weights --experiment_name "gmcnn256x256"

# Vizualise results of full model training

In [0]:
!ls outputs/gmcnn256x256/predicted_pics/wgan/

In [0]:
Image('outputs/gmcnn256x256/predicted_pics/wgan/step_1000.png')

In [0]:
Image('outputs/gmcnn256x256/predicted_pics/wgan/step_2000.png')

# Create zip file with model results and checkpoints

In [0]:
!zip -r outputs.zip outputs

In [0]:
ls

In [0]:
!rm -rf outputs/

In [0]:
!python inpainting-gmcnn-keras/runner.py --train_path images --mask_path mask -warm_up_generator -from_weights

In order to download model result go to: Files -> content, right click on outputs.zip -> Download